In [1]:
import pandas as pd
from pyvi import ViTokenizer
import re
import nltk

# 1. Import data

In [14]:
filename = "train_nor_811.xlsx"
data = pd.read_excel(filename, engine = "openpyxl")

In [15]:
data.head()

,Unnamed: 0,Emotion,Sentence
0,188,Other,cho mình xin bài nhạc tên là gì với ạ
1,166,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2,1345,Disgust,lo học đi . yêu đương lol gì hay lại thích học...
3,316,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này :))
4,1225,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...


In [16]:
data.drop(columns = {"Unnamed: 0"}, axis = 1, inplace = True)
data.head()

,Emotion,Sentence
0,Other,cho mình xin bài nhạc tên là gì với ạ
1,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2,Disgust,lo học đi . yêu đương lol gì hay lại thích học...
3,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này :))
4,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...


In [17]:
from sklearn.preprocessing import LabelEncoder
data["emotion_encode"] = data["Emotion"]
encoder = LabelEncoder()
data.emotion_encode = encoder.fit_transform(data.Emotion)
data.head()

,Emotion,Sentence,emotion_encode
0,Other,cho mình xin bài nhạc tên là gì với ạ,4
1,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...,1
2,Disgust,lo học đi . yêu đương lol gì hay lại thích học...,1
3,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này :)),2
4,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...,2


# 2. Data visualization

# 3. Data preprocessing

In [18]:
clean_sentences = []
for i in range(len(data)):
    clean_sentences.append(ViTokenizer.tokenize(data.Sentence[i]))

In [19]:
import json
f = open("sensitive_words.json", encoding="utf-8")
sensitive_words = json.load(f)
f.close()

In [8]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def normalize_sentences(sentences):
    punc_lst = {'.', ',', '...', '-', '“', '”', ':', '(', ')', '"', '!', '&', ';', '?', '*', ']', '>', '…', '’',"``","''"}
    clean_sentences = []
    
    # remove punctuation and lowercase
    for sent in sentences:
        
        # remove emojis
        sent = deEmojify(sent)
        
        sent = nltk.word_tokenize(sent)
        temp = []
        for word in sent:
            word = word.lower()
            if (word in punc_lst or word in sensitive_words):
                continue
            elif(word.isdigit()):
                temp.append("<NUM>")
            else:
                temp.append(word)
        # remove whitespace
        sent = ' '.join(temp)
        
        clean_sentences.append(sent)
        
    return clean_sentences

In [20]:
clean_sentences = normalize_sentences(clean_sentences)

# 4. Model architecture

In [10]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras import layers , activations , models , preprocessing , utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [21]:
MAX_LEN = 150

tokenizer = Tokenizer(lower = True, split = ' ')
tokenizer.fit_on_texts(clean_sentences)

X = tokenizer.texts_to_sequences(clean_sentences)

X = pad_sequences(X, MAX_LEN, padding='post', truncating='post')

print(X.shape)

(5548, 150)


In [22]:
vocab_size = len(tokenizer.word_index) + 1

In [24]:
from  sklearn.model_selection import train_test_split
Y = tf.keras.utils.to_categorical(data.emotion_encode)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 36)

## 4.1 CNN Model

In [30]:
embedding_dim = 128

inputs = layers.Input(shape=( MAX_LEN , ))
embedding = layers.Embedding(vocab_size, embedding_dim, input_length=MAX_LEN)(inputs)

cnn1 = layers.Conv1D(filters=100, kernel_size=1, activation='relu')(embedding)
cnn1 = layers.MaxPooling1D(pool_size=2)(cnn1)
cnn1 = Flatten()(cnn1)

cnn2 = layers.Conv1D(filters=100, kernel_size=2, activation='relu')(embedding)
cnn2 = layers.MaxPooling1D(pool_size=2)(cnn2)
cnn2 = Flatten()(cnn2)

outputs = layers.Concatenate()([cnn1,cnn2])

outputs = layers.Dense(28, activation='tanh')(outputs)
outputs = layers.Dense(14, activation='tanh')(outputs)
outputs = layers.Dense(7, activation='softmax')(outputs)
model=models.Model(inputs,outputs)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 150, 128)     521344      input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 150, 100)     12900       embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 149, 100)     25700       embedding_1[0][0]                
_______________________________________________________________________________________

In [31]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1)
history = model.fit(X_train, y_train,
                    epochs=10,
                    callbacks = [es],
                    validation_data=(X_test, y_test),
                    batch_size=64)

Epoch 1/10
66/66 [==============================] - 6s 97ms/step - loss: 0.3811 - accuracy: 0.2850 - val_loss: 0.3804 - val_accuracy: 0.2920
Epoch 2/10
66/66 [==============================] - 6s 88ms/step - loss: 0.3447 - accuracy: 0.4095 - val_loss: 0.3459 - val_accuracy: 0.4009
Epoch 3/10
66/66 [==============================] - 6s 94ms/step - loss: 0.2689 - accuracy: 0.6011 - val_loss: 0.3248 - val_accuracy: 0.4600
Epoch 4/10
66/66 [==============================] - 6s 91ms/step - loss: 0.1862 - accuracy: 0.7688 - val_loss: 0.3388 - val_accuracy: 0.4679
Epoch 00004: early stopping


## 4.2 LSTM Model

In [36]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

model2 = Sequential()
model2.add(Embedding(vocab_size, embedding_dim, input_length=MAX_LEN))
model2.add(SpatialDropout1D(0.2))
model2.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(7, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
history2 = model.fit(X_train, y_train,
                    epochs=10,
                    callbacks = [es],
                    validation_data=(X_test, y_test),
                    batch_size=32)

Epoch 1/10
131/131 [==============================] - 8s 60ms/step - loss: 0.0265 - accuracy: 0.9813 - val_loss: 0.4944 - val_accuracy: 0.4340
Epoch 2/10
131/131 [==============================] - 7s 56ms/step - loss: 0.0147 - accuracy: 0.9923 - val_loss: 0.5181 - val_accuracy: 0.4456
Epoch 00002: early stopping


## 4.3 Decision Tree Model + TF IDF vectorize

In [42]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [38]:
# TF IDF vectorize
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(use_idf = True,max_features=3000)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(clean_sentences)
print(tfidf_vectorizer_vectors.shape)

(5548, 3000)


In [40]:
tree_model = DecisionTreeClassifier()
Y_DT = data.emotion_encode

kf = KFold(n_splits=10)
for train_index, test_index in kf.split(tfidf_vectorizer_vectors):
    X_train_DT, X_test_DT = tfidf_vectorizer_vectors[train_index], tfidf_vectorizer_vectors[test_index]
    y_train_DT, y_test_DT = Y_DT[train_index], Y_DT[test_index]
    
    tree_model.fit(X_train_DT, y_train_DT)

In [41]:
y_pred_DT = tree_model.predict(X_test_DT)

In [43]:
print(classification_report(y_test_DT, y_pred_DT))

              precision    recall  f1-score   support

           0       0.09      0.08      0.08        39
           1       0.28      0.35      0.31       101
           2       0.47      0.43      0.45       173
           3       0.59      0.30      0.40        33
           4       0.31      0.33      0.32       101
           5       0.35      0.44      0.39        82
           6       0.33      0.16      0.22        25

   micro avg       0.35      0.35      0.35       554
   macro avg       0.35      0.30      0.31       554
weighted avg       0.36      0.35      0.35       554

